In [3]:
import warnings
warnings.filterwarnings("ignore")

## Contexte

Les ingredients d'un complément alimentaires sont de différents types dans Teleicare :
* plante
* micro-organisme
* nutriment (forme d'apport de nutriment)
* arôme
* additif
* autre ingrédient actif
* autre ingrédient


**Est-il possible de simplifier la base en associant à chaque type d'ingrédient une activité ?**
*OUI presque*
* plante -> actif
* micro-organisme -> actif
* nutriment (forme d'apport) -> actif
* additif -> non actif
* arôme -> non actif
* autre ingrédient actif -> cela dépend, à clarifier
* autre ingrédient -> cela dépend, à clarifier

**En pratique, dans les déclarations, l'activité des ingrédients est-elle modifiée par rapport à celle "par défaut" indiquée dans les tables de référence ?**


In [132]:
import pandas as pd
import numpy as np

In [5]:
ingredient_df = pd.read_csv("../../csv-data/REF_ICA_INGREDIENT_AUTRE.csv", encoding="utf-16")
ingredient_df.describe()

,INGA_IDENT,STINGSBS_IDENT,TAING_IDENT,FCTINGR_IDENT,INGA_ORDRE
count,1173.000000,1173.000000,1173.000000,1173.000000,0.0
mean,1283.190111,2.488491,3.164535,1.467178,NaN
std,346.466307,1.484589,1.382868,0.499134,NaN
min,691.000000,1.000000,1.000000,1.000000,NaN
25%,984.000000,1.000000,2.000000,1.000000,NaN
50%,1277.000000,2.000000,4.000000,1.000000,NaN
75%,1581.000000,4.000000,4.000000,2.000000,NaN
max,1887.000000,4.000000,5.000000,2.000000,NaN


In [23]:
ACTIVITY = {1: "actif", 2: "non actif"}

TYPE = {1: "Nutriment (Forme d'apport)",
2: "Additif",
3: "Arôme",
4: "Autre ingrédient actif",
5: "Autre ingrédient"}

STATUS = {
1: "autorisé",
2: "non autorisé",
3: "à inscrire",
4: "sans objet",
}

In [24]:
type_and_activity = ingredient_df[['TAING_IDENT', 'FCTINGR_IDENT']]
type_and_activity['type'] = ingredient_df['TAING_IDENT'].apply(lambda x : TYPE[x])
type_and_activity['activity'] = ingredient_df['FCTINGR_IDENT'].apply(lambda x : ACTIVITY[x])
type_and_activity['status'] = ingredient_df['STINGSBS_IDENT'].apply(lambda x : STATUS[x])
type_and_activity

,TAING_IDENT,FCTINGR_IDENT,type,activity,status
0,1,1,Nutriment (Forme d'apport),actif,autorisé
1,1,1,Nutriment (Forme d'apport),actif,autorisé
2,1,1,Nutriment (Forme d'apport),actif,autorisé
3,1,1,Nutriment (Forme d'apport),actif,autorisé
4,1,1,Nutriment (Forme d'apport),actif,autorisé
...,...,...,...,...,...
1168,4,1,Autre ingrédient actif,actif,autorisé
1169,4,1,Autre ingrédient actif,actif,autorisé
1170,4,1,Autre ingrédient actif,actif,autorisé
1171,5,2,Autre ingrédient,non actif,autorisé


### Quelle est l'activité par type d'ingrédient ?

In [8]:
type_and_activity[['type', 'activity']].groupby(['type', 'activity']).value_counts().to_frame()

count
type                       activity        
Additif                    non actif    250
Arôme                      non actif    100
Autre ingrédient           actif          8
                           non actif    197
Autre ingrédient actif     actif        422
                           non actif      1
Nutriment (Forme d'apport) actif        195

In [9]:
# quel est cet "Autre ingrédient actif" qui est non actif ?
ingredient_df[(type_and_activity['activity'] == "non actif") & (type_and_activity['type'] == "Autre ingrédient actif")]

# "Arôme - caramel" => devrait être un arôme

,INGA_IDENT,STINGSBS_IDENT,TAING_IDENT,FCTINGR_IDENT,INGA_LIBELLE,INGA_LIBELLE_EN,INGA_OBSERVATION,INGA_COMMENTAIRE_PUBLIC,INGA_COMMENTAIRE_PUBLIC_EN,INGA_COMMENTAIRE_PRIVE,INGA_COMMENTAIRE_PRIVE_EN,INGA_DESCRIPTION,INGA_DESCRIPTION_EN,INGA_ORDRE,INGA_OBSOLET
861,1562,1,4,2,Arôme - caramel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [10]:
# quel sont ces "Autre ingrédient" qui sont actifs ?
ingredient_df[(type_and_activity['activity'] == "actif") & (type_and_activity['type'] == "Autre ingrédient")]


,INGA_IDENT,STINGSBS_IDENT,TAING_IDENT,FCTINGR_IDENT,INGA_LIBELLE,INGA_LIBELLE_EN,INGA_OBSERVATION,INGA_COMMENTAIRE_PUBLIC,INGA_COMMENTAIRE_PUBLIC_EN,INGA_COMMENTAIRE_PRIVE,INGA_COMMENTAIRE_PRIVE_EN,INGA_DESCRIPTION,INGA_DESCRIPTION_EN,INGA_ORDRE,INGA_OBSOLET
728,1419,1,5,1,hydrolysat de poisson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
809,1509,4,5,1,Caséinate de calcium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
810,1510,4,5,1,Ovalbumine,Egg albumin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
922,1627,4,5,1,Argile Beige,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
923,1628,1,5,1,Sels Thermaux de Vichy,Thermal salts of Vichy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
929,1634,1,5,1,Amidon d''orge cireux,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
949,1661,1,5,1,Chlorophylle,Chlorophyll,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
952,1665,3,5,1,Levure enrichie en vitamine D,Yeast enriched with vitamin D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [28]:
# Activité de plantes et micro-organisme
plant_df = pd.read_csv("../../csv-data/REF_ICA_PLANTE.csv", encoding="utf-16")
plant_df.describe()

moorg_df = pd.read_csv("../../csv-data/REF_ICA_MICRO_ORGANISME.csv", encoding="utf-16")
moorg_df.describe()

,MORG_IDENT,FCTINGR_IDENT,STINGSBS_IDENT,MORG_COMMENTAIRE_PUBLIC_EN,MORG_COMMENTAIRE_PRIVE_EN,MORG_ORDRE
count,91.00000,91.0,91.000000,0.0,0.0,0.0
mean,46.00000,1.0,1.032967,NaN,NaN,NaN
std,26.41338,0.0,0.314485,NaN,NaN,NaN
min,1.00000,1.0,1.000000,NaN,NaN,NaN
25%,23.50000,1.0,1.000000,NaN,NaN,NaN
50%,46.00000,1.0,1.000000,NaN,NaN,NaN
75%,68.50000,1.0,1.000000,NaN,NaN,NaN
max,91.00000,1.0,4.000000,NaN,NaN,NaN


In [43]:
plant_df['activity'] = plant_df['FCTINGR_IDENT'].apply(lambda x : ACTIVITY[x])

plant_df[['activity']].groupby('activity').value_counts().to_frame()
# Toutes les plantes sont actives


,count
activity,
actif,1783


In [44]:
moorg_df['activity'] = moorg_df['FCTINGR_IDENT'].apply(lambda x : ACTIVITY[x])

moorg_df[['activity']].groupby('activity').value_counts().to_frame()# Tous les microorganismes sont actifs

,count
activity,
actif,91


### Quel est le status (autorisé ou non) par type d'ingrédient ?

In [25]:
type_and_activity[['type', 'status']].groupby(['type', 'status']).value_counts().to_frame()

# TODO Est-ce qu'il y a aussi des sans objets dans les Plantes et Microorganismes

count
type                       status             
Additif                    autorisé          3
                           non autorisé      2
                           sans objet      245
Arôme                      autorisé          4
                           sans objet       96
Autre ingrédient           autorisé         19
                           non autorisé      1
                           sans objet      184
                           à inscrire        1
Autre ingrédient actif     autorisé        354
                           non autorisé     21
                           sans objet       47
                           à inscrire        1
Nutriment (Forme d'apport) autorisé        193
                           non autorisé      2

In [50]:
# Les valeurs singulières
ingredient_df[(type_and_activity['status'] == 'non autorisé') & (type_and_activity['type'] == 'Nutriment (Forme d\'apport)')]

,INGA_IDENT,STINGSBS_IDENT,TAING_IDENT,FCTINGR_IDENT,INGA_LIBELLE,INGA_LIBELLE_EN,INGA_OBSERVATION,INGA_COMMENTAIRE_PUBLIC,INGA_COMMENTAIRE_PUBLIC_EN,INGA_COMMENTAIRE_PRIVE,INGA_COMMENTAIRE_PRIVE_EN,INGA_DESCRIPTION,INGA_DESCRIPTION_EN,INGA_ORDRE,INGA_OBSOLET
1007,1720,2,1,1,Soufre,Sulfur,NaN,Le soufre n'est pas autorisé par la directive ...,NaN,NaN,NaN,NaN,NaN,NaN,True
1162,1877,2,1,1,L-thréonate de magnésium,Magnesium L-Threonate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [52]:
ingredient_df[(type_and_activity['status'] == 'non autorisé') & (type_and_activity['type'] == 'Autre ingrédient actif')]

,INGA_IDENT,STINGSBS_IDENT,TAING_IDENT,FCTINGR_IDENT,INGA_LIBELLE,INGA_LIBELLE_EN,INGA_OBSERVATION,INGA_COMMENTAIRE_PUBLIC,INGA_COMMENTAIRE_PUBLIC_EN,INGA_COMMENTAIRE_PRIVE,INGA_COMMENTAIRE_PRIVE_EN,INGA_DESCRIPTION,INGA_DESCRIPTION_EN,INGA_ORDRE,INGA_OBSOLET
355,1046,2,4,1,Agmatine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
356,1047,2,4,1,Clinoptilolite,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
357,1048,2,4,1,Glycocyamine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
358,1049,2,4,1,Indole-3-carbinol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
359,1050,2,4,1,Ipriflavone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
360,1051,2,4,1,Méthylamine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
361,1052,2,4,1,Méthylcellulose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
362,1053,2,4,1,Noréphédrine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
364,1055,2,4,1,Prégnénolone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
365,1056,2,4,1,Sérotonine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [54]:
ingredient_df[(type_and_activity['status'] == 'non autorisé') & (type_and_activity['type'] == 'Additif')]

,INGA_IDENT,STINGSBS_IDENT,TAING_IDENT,FCTINGR_IDENT,INGA_LIBELLE,INGA_LIBELLE_EN,INGA_OBSERVATION,INGA_COMMENTAIRE_PUBLIC,INGA_COMMENTAIRE_PUBLIC_EN,INGA_COMMENTAIRE_PRIVE,INGA_COMMENTAIRE_PRIVE_EN,INGA_DESCRIPTION,INGA_DESCRIPTION_EN,INGA_ORDRE,INGA_OBSOLET
426,1117,2,2,2,E171,E171,NaN,Additif interdit en UE depuis 2022 (https://eu...,NaN,"Groupe II : quantum satis / CA solide, liquide...",NaN,NaN,NaN,NaN,False
1097,1812,2,2,2,E127,E127,NaN,Cet additif n'est pas autorisé dans les complé...,NaN,Ne figure pas dans les additifs autorisés pour...,NaN,Colorant rouge synthétique,NaN,NaN,False


=> certains additifs étant interdits, il faudrait devoir redéclarer à chaque changement d'additifs

In [57]:
ingredient_df[(type_and_activity['status'] == 'autorisé') & (type_and_activity['type'] == 'Arôme')]

,INGA_IDENT,STINGSBS_IDENT,TAING_IDENT,FCTINGR_IDENT,INGA_LIBELLE,INGA_LIBELLE_EN,INGA_OBSERVATION,INGA_COMMENTAIRE_PUBLIC,INGA_COMMENTAIRE_PUBLIC_EN,INGA_COMMENTAIRE_PRIVE,INGA_COMMENTAIRE_PRIVE_EN,INGA_DESCRIPTION,INGA_DESCRIPTION_EN,INGA_ORDRE,INGA_OBSOLET
746,1441,1,3,2,arôme - menthe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
747,1442,1,3,2,Arôme grenadine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
748,1443,1,3,2,Arôme - mangue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1055,1768,1,3,2,arôme clou de girofle,clove aroma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


=> le fait que les arômes soient tous autorisés ou sans objet fait pencher pour dire qu'il n'y a pas besoin de refaire une déclaration pour 2 arômes différents ?
ou permettre une déclaration, qui ne passe pas par le processus d'instruction mais qui vient remplir la BDD de tous les CA disponibles en France

Y-a t'il une nécessité légale à la redéclaration ?

il pourrait y avoir une uniformisaiont de l'écriture des nom d'arômes :
- sans tiret entre 'arôme' et son nom

In [45]:
plant_df['status'] = plant_df['STINGSBS_IDENT'].apply(lambda x : STATUS[x])

plant_df[['status']].groupby('status').value_counts().to_frame()

,count
status,
autorisé,1013
non autorisé,591
à inscrire,179


In [46]:
moorg_df['status'] = moorg_df['STINGSBS_IDENT'].apply(lambda x : STATUS[x])

moorg_df[['status']].groupby('status').value_counts().to_frame()

,count
status,
autorisé,90
sans objet,1


In [47]:
moorg_df[moorg_df['status'] == 'sans objet']

,MORG_IDENT,FCTINGR_IDENT,STINGSBS_IDENT,MORG_ESPECE,MORG_COMMENTAIRE_PUBLIC,MORG_COMMENTAIRE_PUBLIC_EN,MORG_COMMENTAIRE_PRIVE,MORG_COMMENTAIRE_PRIVE_EN,MORG_ORDRE,MORG_OBSOLET,MORG_GENRE,activity,status
84,85,1,4,faecium,NaN,NaN,NaN,NaN,NaN,False,Enterococcus,actif,sans objet


### Faudra t'il nettoyer les Arômes, Additifs, Formes d'apport et Autre ingrédients ?

In [18]:
ingredient_df[(type_and_activity['type'] == "Nutriment (Forme d'apport)")]["INGA_DESCRIPTION"].dropna()
# "INGA_OBSERVATION" est vide pour tous
# "INGA_OBSERVATION_EN" est vide pour tous
# Monométhylsilanetriol à pour synonyme et pour description "Silicium organique" TODO : ne conserver que l'un des deux

910     eaux mères riches en magnésium grâce à l évapo...
1034                                   Silicium organique
1098    Nouvel aliment autorisé - forme synthétique du...
1124    Nouvel ingrédient autorisé puis intégré à l'an...
1134    Le nouvel aliment est le fructoborate de calci...
1149    Spécifications Novel Food : La levure de boula...
1150    Le tartrate-adipate d’hydroxyde de fer (IHAT) ...
Name: INGA_DESCRIPTION, dtype: object

In [98]:
ingredient_df[(type_and_activity['type'] == "Nutriment (Forme d'apport)")]["INGA_LIBELLE"]
# Levure de boulanger (Saccha romyces cerevisiae) traitée par UV à corriger orthographe
# redondant avec Levure enrichie en vitamine D qui lui est de type "Autre ingrédient (normalement non actif)" mais avec une activité "actif"
# TODO: sont-ils tous bien liés à la substance qu'il faut

910     eaux mères riches en magnésium grâce à l évapo...
1034                                   Silicium organique
1098    Nouvel aliment autorisé - forme synthétique du...
1124    Nouvel ingrédient autorisé puis intégré à l'an...
1134    Le nouvel aliment est le fructoborate de calci...
1149    Spécifications Novel Food : La levure de boula...
1150    Le tartrate-adipate d’hydroxyde de fer (IHAT) ...
Name: INGA_DESCRIPTION, dtype: object

In [104]:
pd.set_option('display.max_rows', 500)
## TODO Hypothèse à vérifier : Les autres ingrédients actifs ce sont les ingrédients substance (qui ont leur doublon en tant que substance)
## sauf exception type huile de hareng
## Certaines ["INGA_DESCRIPTION"] sont des CAS number
# triplon Huile de Krill ingr Euphausia superba ingr et Huile de Krill substance
# Lait d'annesse ing et lait d'annesse substance
# miel d''acacia ing et miel d'acacia
# peptide d''élastine ing et peptide d'elastine subst et oligopeptide d'elastine subs
# huile de foie de morues ing et huile de foie de morue ing et huile de foie de morue subs
# Fibres ing et Fibres subst et Fibres d''agrumes ing et Fibres de pommes ing et Fibres d'acacia
# 'huile de TCM (triglycérides à chaînes moyennes)' vs 'Triglycérides à chaine moyenne'
aia_df = ingredient_df[(type_and_activity['type'] == "Autre ingrédient actif")]["INGA_LIBELLE"]

aia_df

184                                       Acétylcarnitine
185                                        Acétylcystéine
186                                      Acétylméthionine
187                       Acide adénosine-5'-phosphorique
188                            Acide alpha-aminobutyrique
189                                      Acide aspartique
190                    Acide cytidine-5'-monophosphorique
191                            Acide gamma-aminobutyrique
192                                      Acide glutamique
193                       Acide guanosine-5'-phosphorique
194                                    Acide hyaluronique
195                         Acide inosine-5'-phosphorique
196                             Acide linoléique conjugué
197                                        Acide lipoïque
198                                        Acide orotique
199                             Acide para-aminobenzoïque
200                                       Acide pyruvique
201           

In [106]:
pd.set_option('display.max_rows', 250)

additif_df = ingredient_df[(type_and_activity['type'] == "Additif")]['INGA_LIBELLE']
additif_df
#  hydroxypropyl méthyl cellulose
#  citrate de calcium
#  vinaigre d''alcool
#  stévia

371                               E100
372                               E101
373                               E102
374                               E104
375                               E110
376                              E1103
377                               E120
378                              E1200
379                              E1201
380                              E1202
381                              E1203
382                              E1204
383                              E1205
384                              E1206
385                              E1207
386                              E1208
387                              E1209
388                               E122
389                               E124
390                               E129
391                               E131
392                               E132
393                               E133
394                               E140
395                              E1404
396                      

In [111]:
pd.set_option('display.max_rows', 10)

def remove_prefix(name):
    return name.lower().replace("arôme", "").replace("-","").strip()

arome_df = ingredient_df[(type_and_activity['type'] == "Arôme")]['INGA_LIBELLE'].apply(lambda x: remove_prefix(x))
arome_df.value_counts()
# les arômes grenadine et agrumes sont présents 2 fois

INGA_LIBELLE
grenadine     2
agrumes       2
café          1
tiramisu      1
thym          1
             ..
kiwi          1
canneberge    1
abricot       1
biscuit       1
violette      1
Name: count, Length: 98, dtype: int64

In [110]:
# TODO: 1 seule description pour l'arome réglisse dont la description est "arôme"

In [77]:
pd.set_option('display.max_rows', 100)
arome_df.to_frame().sort_values(by=["INGA_LIBELLE"])
# naturel  citron vs citron
# fruits rouges et fruit rouge
# bergamote et de bergamote
# ethylvanilline vs vanille vs vanilline
# raisin blanc vs raisin rouge
# "arômes" https://compl-alim.beta.gouv.fr/element/956--ingredient--Ar%C3%B4mes
# et "arôme" https://compl-alim.beta.gouv.fr/element/798--ingredient--Ar%C3%B4me
# et "arôme naturel" https://compl-alim.beta.gouv.fr/element/739--ingredient--Ar%C3%B4me%20naturel

,INGA_LIBELLE
797,
780,abricot
843,acérola
936,agrumes
901,agrumes
948,amande
761,ananas
751,anis
888,baies
834,banane


### Activité spécifiée dans les déclarations

In [116]:
# En pratique, comment est utilisé cette possibilité d'activité/non activité dans les déclarations

ingr_declare_df = pd.read_csv("../../decla-data/ICA_INGREDIENT.csv", encoding="utf-16")
ingr_declare_df[['TYING_IDENT', 'FCTINGR_IDENT']].groupby(['TYING_IDENT', 'FCTINGR_IDENT']).value_counts().to_frame()
# les types correspondants sont dans REF_ICA_TYPE_INGREDIENT

TYPE_INGR_DECLA = {
    1: "Plantes",
    2: "Micro-organismes",
    3: "Autres ingrédients"
}

count
TYING_IDENT FCTINGR_IDENT        
1           1              202321
            2                5448
2           1               15859
            2                  47
3           1              233903
            2              300774

In [114]:
ingr_declare_df

,INGR_IDENT,VRSDECL_IDENT,FCTINGR_IDENT,TYING_IDENT,INGR_COMMENTAIRES
0,1,4,2,3,NaN
1,2,4,1,3,NaN
2,3,4,1,3,NaN
3,4,4,1,3,NaN
4,5,4,1,3,NaN
...,...,...,...,...,...
758347,759764,167827,1,1,NaN
758348,759765,167827,1,1,NaN
758349,759766,167827,1,1,NaN
758350,759767,167827,1,1,NaN


In [118]:
jointure_ingr_declare_ingr_reference = pd.read_csv("../../decla-data/ICA_INGREDIENT_AUTRE.csv", encoding="utf-16")
jointure_ingr_declare_ingr_reference
# il y a aussi une jointure pour les plantes et les micro-organismes ?
# ICA_MICRO_ORGANISME pour les Moorg
# ICA_PREPARATION pour les Plantes

,INGR_IDENT,INGA_IDENT
0,5955,691
1,6016,691
2,6053,691
3,14634,691
4,16501,691
...,...,...
534672,732764,1885
534673,750470,1886
534674,751309,1887
534675,751350,1887


In [120]:
declaration_with_ingredient_id_df = jointure_ingr_declare_ingr_reference.merge(ingr_declare_df, on="INGR_IDENT", how="left") # on INGR_IDENT
declaration_with_ingredient_id_df

,INGR_IDENT,INGA_IDENT,VRSDECL_IDENT,FCTINGR_IDENT,TYING_IDENT,INGR_COMMENTAIRES
0,5955,691,1827,1,3,NaN
1,6016,691,1831,1,3,NaN
2,6053,691,1897,1,3,NaN
3,14634,691,4104,1,3,NaN
4,16501,691,6464,1,3,NaN
...,...,...,...,...,...,...
534672,732764,1885,160956,1,3,NaN
534673,750470,1886,164205,2,3,NaN
534674,751309,1887,165472,1,3,NaN
534675,751350,1887,165470,1,3,NaN


In [122]:
ingredient_df

,INGA_IDENT,STINGSBS_IDENT,TAING_IDENT,FCTINGR_IDENT,INGA_LIBELLE,INGA_LIBELLE_EN,INGA_OBSERVATION,INGA_COMMENTAIRE_PUBLIC,INGA_COMMENTAIRE_PUBLIC_EN,INGA_COMMENTAIRE_PRIVE,INGA_COMMENTAIRE_PRIVE_EN,INGA_DESCRIPTION,INGA_DESCRIPTION_EN,INGA_ORDRE,INGA_OBSOLET
0,691,1,1,1,5'-déoxyadénosylcobalamine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,692,1,1,1,6-palmitate de L-ascorbyl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,693,1,1,1,Acétate de calcium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,694,1,1,1,Acétate de D-alpha-tocophéryle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,695,1,1,1,Acétate de DL-alpha-tocophéryle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1168,1883,1,4,1,isoflavones issues de Glycine max. (L.) Merr.,isoflavones,NaN,NaN,NaN,https://www.efsa.europa.eu/fr/press/news/151021,NaN,NaN,NaN,NaN,False
1169,1884,1,4,1,acide gamma-linolénique,gamma-linolenic acid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1170,1885,1,4,1,Huile de hareng,Herring oil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1171,1886,1,5,2,Lactose anhydre,Anhydrous Lactose,NaN,NaN,NaN,Utilisation du lactose anhydre dans le domaine...,NaN,NaN,NaN,NaN,False


In [128]:
decla_with_ingredient = declaration_with_ingredient_id_df.merge(ingredient_df, on="INGA_IDENT", how="left", suffixes=("_declared", "_in_reference")) # on INGA_IDENT
decla_with_ingredient

,INGR_IDENT,INGA_IDENT,VRSDECL_IDENT,FCTINGR_IDENT_declared,TYING_IDENT,INGR_COMMENTAIRES,STINGSBS_IDENT,TAING_IDENT,FCTINGR_IDENT_in_reference,INGA_LIBELLE,INGA_LIBELLE_EN,INGA_OBSERVATION,INGA_COMMENTAIRE_PUBLIC,INGA_COMMENTAIRE_PUBLIC_EN,INGA_COMMENTAIRE_PRIVE,INGA_COMMENTAIRE_PRIVE_EN,INGA_DESCRIPTION,INGA_DESCRIPTION_EN,INGA_ORDRE,INGA_OBSOLET
0,5955,691,1827,1,3,NaN,1,1,1,5'-déoxyadénosylcobalamine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,6016,691,1831,1,3,NaN,1,1,1,5'-déoxyadénosylcobalamine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,6053,691,1897,1,3,NaN,1,1,1,5'-déoxyadénosylcobalamine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,14634,691,4104,1,3,NaN,1,1,1,5'-déoxyadénosylcobalamine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,16501,691,6464,1,3,NaN,1,1,1,5'-déoxyadénosylcobalamine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534672,732764,1885,160956,1,3,NaN,1,4,1,Huile de hareng,Herring oil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
534673,750470,1886,164205,2,3,NaN,1,5,2,Lactose anhydre,Anhydrous Lactose,NaN,NaN,NaN,Utilisation du lactose anhydre dans le domaine...,NaN,NaN,NaN,NaN,False
534674,751309,1887,165472,1,3,NaN,1,1,1,Sulfate de magnésium,magnesium sulfate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
534675,751350,1887,165470,1,3,NaN,1,1,1,Sulfate de magnésium,magnesium sulfate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [136]:
decla_with_ingredient[["INGA_IDENT", "FCTINGR_IDENT_declared", "FCTINGR_IDENT_in_reference"]]
decla_with_ingredient['FCTINGR_IDENT_declared'].equals(decla_with_ingredient['FCTINGR_IDENT_in_reference'])
# les 2 colonnes ne sont pas égales

False

In [148]:
decla_with_ingredient['same_activity'] = np.where((decla_with_ingredient['FCTINGR_IDENT_declared'] == decla_with_ingredient['FCTINGR_IDENT_in_reference']), True, False)
len(decla_with_ingredient[decla_with_ingredient['same_activity'] == False]['INGA_LIBELLE'].unique())
# 518 ingrédients n'ont pas la même activité déclarée que celle dans la table de référence

518

In [149]:
decla_with_ingredient[decla_with_ingredient['same_activity'] == False]['INGA_LIBELLE'].unique()

array(['6-palmitate de L-ascorbyl', 'Acétate de D-alpha-tocophéryle',
       'Acétate de DL-alpha-tocophéryle', 'Acétate de rétinyle',
       'Acétyl-taurinate de magnésium',
       'acide (6S)-5-méthyltétrahydrofolique sous forme de sel de glucosamine',
       'Acide L-ascorbique', 'Acide ptéroylmonoglutamique',
       'Acide silicique', 'Ascorbate de manganèse', 'Béta-carotène',
       'Bicarbonate de potassium', 'Bicarbonate de sodium',
       'Bisglycinate de calcium', 'Bisglycinate de magnésium',
       'Bisglycinate de zinc', 'Borate de sodium', 'Carbonate de calcium',
       'Carbonate de cuivre', 'Carbonate de magnésium',
       'Carbonate de manganèse', 'Carbonate de potassium',
       'Carbonate de sodium', 'Carbonate de zinc',
       'Chlorhydrate de pyridoxine', 'Chlorhydrate de thiamine',
       'Chlorure de chrome', 'Chlorure de magnésium',
       'Chlorure de potassium', 'Chlorure de sodium', 'Cholécalciférol',
       'Citrate de manganèse', 'Citrate de potassium',
     

À discuter avec BEPIAS : est-ce que c'est vraiment pertinent de laisser les déclarants choisir l'activité des ingrédients ?
Si un ingrédient est reconnu actif, pourquoi deviendrait-il soudainement inactif dans un certain type de préparation ?

In [ ]:
# le type de l'ing déclaré TYING_IDENT et le type de l'ing de référence TAING_IDENT sont différents 
# TAING_IDENT est plus spécifique
decla_with_ingredient['type'] = decla_with_ingredient['TAING_IDENT']

In [56]:
decla_with_ingredient['type'] = 


,INGR_IDENT,INGA_IDENT
0,5955,691
1,6016,691
2,6053,691
3,14634,691
4,16501,691
...,...,...
534672,732764,1885
534673,750470,1886
534674,751309,1887
534675,751350,1887
